# AdImpactSimulation

This is a simple simulation of the impact of brand and targeted direct response advertising
on purchase decisions.

## User Needs

People (users) have "needs" for products and services which change over time.
Needs for products that are consumed regularly, like food and shampoo, increase as
existing products are consumed.  Needs for other products, like new electronic devices,
only develop as consumers learn about the products and how they might be useful.
Advertising can help make people aware of new products and ways that they might be helpful,
and this can increase the need for those products.

We'll start by using the same needs for all users.  Later, we might want different
levels of needs for different demographic groups.

Our similation will look at needs and advertising over time in weekly units of time.


In [2]:
# Time unit
time_unit_name = "week"

# General time-based need for a product.
# A typical user's need will increase by about this much each week, regardless on advertising.
time_based_need = 1 / 52

# General brand advertising-based need for a product.
# One ad for a product in the past week will result in this much need for a typical user
# who may be likely to purchase the product.
# (Some products are only appropriate for a subset of the population.)
brand_based_need = 1 / 100

## Brands

The simulations will assume that there are multiple brands producing competing products.
Each brand can have difference levels of brand and direct response advertising and
different search rankings (which might include natural or paid search).

We'll start by using the same advertising levels for all users.  Later, we might want different
levels of advertising for different demographic groups.

In [3]:
import random

# The number of advertisers / product manufacturers in the simulation
advertiser_count = 4

# The names of the advertisers / product manufacturers
advertiser_names = ["Advertiser " + str(i + 1) for i in range(advertiser_count)]

# The expected number of brand ads from each advertiser in a week
advertiser_brand_frequency = {
    adv : random.triangular(low = 0.2, high = 2.0, mode = 1.0)
    for adv in advertiser_names
}

# The expected number of trageted ads (to targeted users) from each advertiser in a week
advertiser_targeted_frequency = {
    adv : random.triangular(low = 0.2, high = 2.0, mode = 1.0)
    for adv in advertiser_names
}

# The search positions of the advertisers
advertiser_search_position = { adv : i + 1 for i, adv in enumerate(advertiser_names)}

# Brand dimensions
# Some users prefer low price or high quality, ease of use, or other factors.
# Users will be more likely to buy products that match the values that are most important to them.
brand_dimensions = ['Price', 'Quality', 'Ease of Use']
brand_dimension_advertiser_ratings = {
    brand_dim : {
        adv : random.random()
        for adv in advertiser_names
    }
    for brand_dim in brand_dimensions
}

## Advertising Impact

The impact of advertising diminishes as advertising increases and over time.
For example, the impact of 10 ads to a user in a week may be less than 10 times
the impact of 1 ad to a user in a week, and the impact of an ad 2 weeks ago is probably
less than the impact of an ad 1 week ago.

We use saturation functions to model the weekly ad saturation effect.  The decrease in ad
impact over time is modeled by an exponential decay with a half-life that specifies the number
of weeks when impact of the ads will decrease by one half.

In [5]:
# General sturation function
# No saturation when the weekly ad count is not more than saturation_start.
# Saturation with additional ads so that the total returned is never more than
# saturation_start + saturation_range
def ad_saturation(ad_count, saturation_start = 4, saturation_range = 6):
    if ad_count <= saturation_start:
        return ad_count
    adjusted_ad_count = (ad_count - saturation_start) / saturation_range
    return saturation_start + saturation_range * (adjusted_ad_count / (adjusted_ad_count + 1))


# Brand sturation function
# No saturation when the weekly ad count is not more than saturation_start.
# Saturation with additional ads so that the total returned is never more than
# saturation_start + saturation_range
def weekly_brand_saturation(ad_count, saturation_start = 4, saturation_range = 6):
    return ad_saturation(
        ad_count,
        saturation_start = saturation_start,
        saturation_range = saturation_range
    )

# Targerted sturation function
# No saturation when the weekly ad count is not more than saturation_start.
# Saturation with additional ads so that the total returned is never more than
# saturation_start + saturation_range
def weekly_targeted_saturation(ad_count, saturation_start = 4, saturation_range = 6):
    return ad_saturation(
        ad_count,
        saturation_start = saturation_start,
        saturation_range = saturation_range
    )


# Brand half-life
# Brand messages are half as effective after this many weeks
brand_half_life_weeks = 12

# Targerted half-life
# Targerted messages are half as effective after this many weeks
targeted_half_life_weeks = 2